<a href="https://colab.research.google.com/github/erdogan98/DATA_SCIENCE/blob/main/DT_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import scipy
import itertools
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter
from random import choice
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/poster_presentation/dataset/train.csv')

In [ ]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-855eeff3-2bb7-c217-f95e-99be4b7be740)


In [ ]:
data.shape

(159571, 8)

In [ ]:
#preprocess
import os
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import  stopwords
import string
import re
    
data['length'] = data['comment_text'].str.len()

# Convert all messages to lower case
data['comment_text'] = data['comment_text'].str.lower()


# Replace email addresses with 'email'
data['comment_text'] = data['comment_text'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
data['comment_text'] = data['comment_text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
data['comment_text'] = data['comment_text'].str.replace(r'£|\$', 'dollars')

    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
data['comment_text'] = data['comment_text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumber')


# Replace numbers with 'numbr'
data['comment_text'] = data['comment_text'].str.replace(r'\d+(\.\d+)?', 'numbr')




data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in string.punctuation))


stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))


lem=WordNetLemmatizer()
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join(
 lem.lemmatize(t) for t in x.split()))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
#tokenization
tokenized_text = data['comment_text'].apply(lambda x: x.split())

tokenized_text.head()

0    [explanation, edits, made, username, hardcore,...
1    [d'aww!, match, background, colour, i'm, seemi...
2    [hey, man,, i'm, really, trying, edit, war., g...
3    [can't, make, real, suggestion, improvement, w...
4    [you,, sir,, hero., chance, remember, page, th...
Name: comment_text, dtype: object

In [ ]:
data['clean_length'] = data.comment_text.str.len()
data.head(3)

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,length,clean_length
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,264,180
1,000103f0d9cfb60f,d'aww! match background colour i'm seemingly s...,0,0,0,0,0,0,112,111
2,000113f07ec002fd,"hey man, i'm really trying edit war. guy const...",0,0,0,0,0,0,233,149


In [ ]:
#splitting data into test&train
train = data.sample(frac=0.8)
test = data.sample(frac=0.2)

In [ ]:
#Term Frequency-Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=10,stop_words='english')
features=tfidf.fit_transform(train['comment_text'])

x = features
#test_tfidf_matrix=tfidf.fit_transform(test['comment_text'])

#train_df_tfidf = pd.DataFrame(train_tfidf_matrix.todense())
#test_df_tfidf = pd.DataFrame(test_tfidf_matrix.todense())

In [ ]:
x.shape

(127657, 10)

In [ ]:
malignant=train['malignant']
highly_malignant=train['highly_malignant']
rude = train['rude']
threat=train['threat']
abuse=train['abuse']
loathe=train['loathe']

33836     0
45680     0
107138    0
159189    0
66773     0
         ..
19263     0
78225     0
146431    0
47747     0
135504    0
Name: threat, Length: 127657, dtype: int64


In [ ]:
#splitting the data into train and validation

from sklearn.model_selection import train_test_split

#TF-IDF features

x_train, x_valid, y_train, y_valid = train_test_split(x,malignant,test_size=0.3)
#x_train, x_valid, y_train, y_valid = train_test_split(x,highly_malignant,test_size=0.3)
#x_train, x_valid, y_train, y_valid = train_test_split(x,rude,test_size=0.3)
#x_train, x_valid, y_train, y_valid = train_test_split(x,threat,test_size=0.3)
#x_train, x_valid, y_train, y_valid = train_test_split(x,abuse,test_size=0.3)
#x_train, x_valid, y_train, y_valid = train_test_split(x,loathe,test_size=0.3)

In [ ]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [ ]:
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of malignant with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

Test accuracy of malignant with decision tree is 0.8992114470729542
[[34407   178]
 [ 3682    31]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     34585
           1       0.15      0.01      0.02      3713

    accuracy                           0.90     38298
   macro avg       0.53      0.50      0.48     38298
weighted avg       0.83      0.90      0.86     38298



In [ ]:
print('Test accuracy of malignant with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of malignant with kNN is 0.9020314376729855
[[34532    53]
 [ 3699    14]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     34585
           1       0.21      0.00      0.01      3713

    accuracy                           0.90     38298
   macro avg       0.56      0.50      0.48     38298
weighted avg       0.84      0.90      0.86     38298



In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,rude,test_size=0.3)
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of highly_malignant with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

Test accuracy of highly_malignant with decision tree is 0.9898689226591467
[[37910     8]
 [  380     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     37918
           1       0.00      0.00      0.00       380

    accuracy                           0.99     38298
   macro avg       0.50      0.50      0.50     38298
weighted avg       0.98      0.99      0.99     38298



In [ ]:
print('Test accuracy of highly_malignant with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of highly_malignant with kNN is 0.9900778108517416
[[37918     0]
 [  380     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     37918
           1       0.00      0.00      0.00       380

    accuracy                           0.99     38298
   macro avg       0.50      0.50      0.50     38298
weighted avg       0.98      0.99      0.99     38298



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,rude,test_size=0.3)
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of rude with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

Test accuracy of rude with decision tree is 0.9462896234790329
[[36235    68]
 [ 1989     6]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     36303
           1       0.08      0.00      0.01      1995

    accuracy                           0.95     38298
   macro avg       0.51      0.50      0.49     38298
weighted avg       0.90      0.95      0.92     38298



In [ ]:
print('Test accuracy of rude with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of rude with kNN is 0.9477779518512716
[[36296     7]
 [ 1993     2]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     36303
           1       0.22      0.00      0.00      1995

    accuracy                           0.95     38298
   macro avg       0.59      0.50      0.49     38298
weighted avg       0.91      0.95      0.92     38298



In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,threat,test_size=0.3)
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of threat with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

Test accuracy of threat with decision tree is 0.99712778735182
[[38188     0]
 [  110     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38188
           1       0.00      0.00      0.00       110

    accuracy                           1.00     38298
   macro avg       0.50      0.50      0.50     38298
weighted avg       0.99      1.00      1.00     38298



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Test accuracy of threat with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of threat with kNN is 0.99712778735182
[[38188     0]
 [  110     0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38188
           1       0.00      0.00      0.00       110

    accuracy                           1.00     38298
   macro avg       0.50      0.50      0.50     38298
weighted avg       0.99      1.00      1.00     38298



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,abuse,test_size=0.3)
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of abuse with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

print('Test accuracy of abuse with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of abuse with decision tree is 0.9498668337772207
[[36373    63]
 [ 1857     5]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     36436
           1       0.07      0.00      0.01      1862

    accuracy                           0.95     38298
   macro avg       0.51      0.50      0.49     38298
weighted avg       0.91      0.95      0.93     38298

Test accuracy of abuse with kNN is 0.9512768290772364
[[36432     4]
 [ 1862     0]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     36436
           1       0.00      0.00      0.00      1862

    accuracy                           0.95     38298
   macro avg       0.48      0.50      0.49     38298
weighted avg       0.91      0.95      0.93     38298



In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,loathe,test_size=0.3)
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier(criterion='entropy')
dct.fit(x_train,y_train)
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
#predicting the probabilities
dct_prediction = dct.predict(x_valid)
knn_prediction = knn.predict(x_valid)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc

print('Test accuracy of loathe with decision tree is {}'.format(accuracy_score(y_valid,dct_prediction)))
print(confusion_matrix(y_valid,dct_prediction))
print(classification_report(y_valid,dct_prediction))

print('Test accuracy of loathe with kNN is {}'.format(accuracy_score(y_valid,knn_prediction)))
print(confusion_matrix(y_valid,knn_prediction))
print(classification_report(y_valid,knn_prediction))

Test accuracy of loathe with decision tree is 0.9918794715128727
[[37986    12]
 [  299     1]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     37998
           1       0.08      0.00      0.01       300

    accuracy                           0.99     38298
   macro avg       0.53      0.50      0.50     38298
weighted avg       0.99      0.99      0.99     38298

Test accuracy of loathe with kNN is 0.9921666927776908
[[37998     0]
 [  300     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     37998
           1       0.00      0.00      0.00       300

    accuracy                           0.99     38298
   macro avg       0.50      0.50      0.50     38298
weighted avg       0.98      0.99      0.99     38298



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
